### Quality facotr investing and other factors {factor investment}

In [ ]:
import pandas as pd

class PreProcessing:
  def __init__(self, RevDF, product_df):
    # self.YrRev_Dict = {d['Year']:d['Rev'] for d in RevDF}
    self.YrRev_DictList = RevDF[['Year','Rev']].to_dict(orient='records')
    self.products = product_df

  def ProductProcessing(self):
    self.product_id_list = self.products['ProductID']
    initial_inventory = self.products[['Product', 'Initial_Inventory']].to_dict(orient='records')
    self.initial_inventory_dict = {d['Product']:d['Initial_Inventory'] for d in initial_inventory}
    self.product_cost_info = self.products[['Product', 'Vendor','Spread',
                                            'Unit_Price', 'Unit_Cost',
                                            'PerInvoiceRng_Min',
                                            'PerInvoiceRng_Max']].sort_values(by='Spread', ascending=False)
    Products = self.product_cost_info
    list_yrlyProduct_data = []
    for YrRev in self.YrRev_DictList:
      Year = YrRev['Year']
      Rev = YrRev['Rev']
      for product in self.product_cost_info['Product']:
        self.Processed_list = Products[Products['Product']==product].copy()
        spread = self.Processed_list['Spread'].values[0]
        unit_price = self.Processed_list['Unit_Price'].values[0]
        unit_cost = self.Processed_list['Unit_Cost'].values[0]
        division_of_rev = Rev * spread
        q_sold = round(division_of_rev / unit_price,0)
        p_sold = q_sold*unit_price
        New_Rev = p_sold
        self.Processed_list.loc[:,'q_sold'] = q_sold
        self.Processed_list.loc[:,'Product_Rev'] = New_Rev
        self.Processed_list.loc[:,'Year'] = Year
        list_yrlyProduct_data.append(self.Processed_list.values.tolist()[0])
    YrlyProduct_Data_df = pd.DataFrame(list_yrlyProduct_data,
                                       columns = ['Product', 'Vendor','Spread',
                                            'Unit_Price', 'Unit_Cost',
                                            'PerInvoiceRng_Min',
                                            'PerInvoiceRng_Max', 'q_sold',
                                            'Total_Product_Rev','Year'])
    return YrlyProduct_Data_df

In [ ]:
import pandas as pd
from datetime import timedelta, datetime
import duckdb as ddb
import random

class CompanyGeneration:
  def __init__(self, Records, yrly_Data):
    self.records = Records
    self.Yearly_Data = yrly_Data

# Combine Everything into one function that will allow me to call different functions inside the same class
# recording finacial statements and additionl assets/equity seperate functions
  def Retail(self):
    PO_df_rows = []
    Sales_df_rows = []
    for yr in self.Yearly_Data['Year'].unique():
      yr_data = self.Yearly_Data[self.Yearly_Data['Year']==yr].copy()
      for product in self.Yearly_Data['Product'].unique():
        product_data = yr_data[yr_data['Product']==product].copy()
        Quantity_Remaining = product_data['q_sold']
        Yr_Start = datetime(yr, 1, 1)
        Yr_End = datetime(yr, 1, 1)
        Product_Cost = product_data['Unit_Cost']
        Selling_Price = product_data['Unit_Price']
        MinRecords = product_data['PerInvoiceRng_Min']
        MaxRecords = product_data['PerInvoiceRng_Max']

        while Quantity_Remaining > 0:
          Order_Date = Yr_Start + timedelta(days=random.randint(0, (Yr_End-Yr_Start).days))
          # Product is Shipped 1-30 days after order
          Shipment_Date = Order_Date + timedelta(days = random.randint(1, 30))
          Records_PerInvoice = random.randint(MinRecords, MaxRecords)
          if Records_PerInvoice > Quantity_Remaining:
            Records_PerInvoice = Quantity_Remaining
          else:
            pass
          Rev = Records_PerInvoice * Selling_Price
          Tcost = Records_PerInvoice * Product_Cost
          Sales_df_rows.append([yr, product, Records_PerInvoice, Order_Date, Shipment_Date, Selling_Price, Product_Cost, Rev, Tcost])
          Quantity_Remaining = Quantity_Remaining - Records_PerInvoice

    self.Sales_df = pd.DataFrame(Sales_df_rows, columns = ["Year","Product", "Quantity",
                                                        "Order_Date", "Shipment_Date",
                                                        "Selling_Price", "Product_Cost",
                                                        "Rev", "Total_cost", "Vendors"])
    cleaned_df = self.Sales_df[['Shipment_Date', 'Rev']].copy()
    cleaned_df.rename(columns={'Shipment_Date': 'Date', 'Rev': 'Amount'}, inplace=True)
    self.RF.JournalEntries_Bulk(cleaned_df,1000,4000,"dr",1010,1000,70,30)
    # 1000 - Cash | 1010 - Accounts Payable | 4000 - Revenue



  def GeneratePOs(self,Inv_Manager):

    for yr in self.Yearly_Data['Year'].unique():
      yr_data = self.Yearly_Data[self.Yearly_Data['Year']==yr].copy()
    pass

In [ ]:
import random
import pandas as pd

class RecordingFunctions:
    def __init__(self, Data_Manager):
        self.Data = Data_Manager

    # Bulk Data Processing
    def JournalEntries_Bulk(self, Clean_df, DebitAcc, CreditAcc, probability=None, val1=0, val2=0, p1=0, p2=0):
        if probability == None:
            pass
        elif probability.lower() == 'dr':
            DebitAcc = random.choices

        df_debits = Clean_df[['Date','Amount']].copy()
        df_debits['Account'] = DebitAcc
        df_debits['Balance'] = "Debit"
        self.Data.ConcatData('journal',df_debits)

        df_credits = Clean_df[['Date','Amount']].copy()
        df_credits['Account'] = CreditAcc
        df_credits['Balance'] = "Credit"
        self.Data.ConcatData('journal',df_credits)

    # Row by Row Data Processing
    def JournalEntries_Single(self, Date, Amount, DebitAcc, CreditAcc):
        self.General_Journal = pd.DataFrame({"Date":[],"Amount":[],"Account":[],"Balance":[]})
        if isinstance(Amount, list) and isinstance(DebitAcc, list) and isinstance(CreditAcc, list):
            if len(Amount) == len(DebitAcc) + len(CreditAcc):
                len_ofDr = len(DebitAcc)
                DrAmounts = Amount[:len_ofDr]
                CrAmounts = Amount[len_ofDr:]
                Debits = zip(DebitAcc,DrAmounts)
                Credits = zip(CreditAcc,CrAmounts)
                for dr, amount in Debits:
                    Journal_Entry = [Date, dr, amount, "Debit"]
                    self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry
                for cr, amount in Credits:
                    Journal_Entry = [Date, cr, amount, "Credit"]
                    self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry
            else:
                print("ERROR Creating Journal Entries - Incorrect Number of Amounts")
        elif isinstance(Amount, int) or isinstance(Amount, float):
            Journal_Entry = [Date,DebitAcc, Amount, "Debit"]
            self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry
            Journal_Entry = [Date, CreditAcc, Amount, "Credit"]
            self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry
        else:
            print("ERROR Creating Journal Entry - Incorrect DataTypes")
        self.Data.ConcatData('journal',self.General_Journal)

    def Rev_Record(self,Year,Rev):
        fil = 'rev'
        self.Data.ConcatData('rev',Rev,Year)

    def Rev_Request(self,Year):
        fil = 'rev'
        self.Data.RequestData('rev',Year)

class Inventory_manager:
    def __init__(self,Inv=None):
        self.MasterInventory = {} if Inv == None else Inv

    def ReadInventory(self,product):
        return self.MasterInventory[product]

    def UpdateInventory(self,product, New_Inv):
        self.MasterInventory[product] = New_Inv

In [ ]:
import pandas as pd
import random
from datetime import timedelta
from datetime import datetime
import duckdb as ddb

# This is a sample of what a product sheet would look like
# ID for vendor will mean something such as type of product or payment type
product_df = pd.DataFrame({
    "ProductID": [1,2,3,4],
    "Product": ["eggs","wine","cheese","candles"],
    "Vendor": [3001, 3002, 3003, 3004],
    "Unit_Price": [3.20, 16.20, 5.20,20.5],
    "Unit_Cost": [1.20, 3.40, 2.10, 10],
    "Initial_Inventory": [0,0,0,0],
    "Spread":[.30, .50, .10, .10],
    "PerInvoiceRng_Min":[10,10,10,10],
    "PerInvoiceRng_Max":[20,20,20,20]
})

# This would all come in as JSON not created in Python
Initial_Year = 2010
YearsIn_Operation = 10
first_year_rev = 10000000
trend = .05
modjulation = .01

x = first_year_rev
y = Initial_Year
coin = (1,2)

list_years = []

for year in range(YearsIn_Operation):
  Revenue = x
  Year = y
  Trended = Revenue * (1+trend)
  direction = random.sample(coin,1)
  if direction == 1:
    Modjulated = Trended * (1+modjulation)
  else:
    Modjulated = Trended * (1-modjulation)
  list_year_item = [Year, round(Revenue,2)]
  list_years.append(list_year_item)
  x = Modjulated
  y = y+1

# This is what the Data would look like in JSON
rev_df = pd.DataFrame(list_years,columns=["Year","Rev"])


#--------------------------------------------------------------------------------

Yrly_Data = PreProcessing(rev_df, product_df).ProductProcessing()
sales_df = 1 # Sales Generator
po_df = 2 # PO Generator


In [ ]:
PreProcessing(rev_df,product_df).ProductProcessing()